In [29]:
!pip install rdflib

In [184]:
import rdflib

# Load the RDF file
rdf_file = "/Users/chengluotong/Desktop/mapping.rdf" 
graph = rdflib.Graph()
graph.parse(rdf_file)

<Graph identifier=Ne3c42ec5ced54644ab62257336f2d337 (<class 'rdflib.graph.Graph'>)>

In [126]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?VisualFeature ?IIIF_Image
WHERE {
  ?expression cidoc-crm:P165_incorporates ?visualitem .  
  ?visualitem rdf:type cidoc-crm:E36_Visual_Item .
  ?visualitem rdfs:label ?VisualFeature .
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item cidoc-crm:P137_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image 
}
"""

results = graph.query(sparql_query)

for result in results:
    identifier = result["Identifier"]
    title = result["Title"]
    visual_feature = result["VisualFeature"]
    iiif_image = result["IIIF_Image"]

    print(f"Identifier: {identifier}")
    print(f"Title: {title}")
    print(f"Visual Feature: {visual_feature}")
    print(f"IIIF Image: {iiif_image}")

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/2791
Title: Wereldkaart 1509
Visual Feature: vier windhoofden
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/2791/full/full/0/iiifimg2791


In [153]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Identifier ?Title ?Description ?Language ?IIIF_Image 
WHERE {
  ?expression cidoc-crm:P165_incorporates ?linguisticItem .  
  ?linguisticItem rdf:type cidoc-crm:E33_Linguistic_Object .
  ?linguisticItem rdfs:label ?Description .
    OPTIONAL { ?linguisticItem cidoc-crm:P72_has_language ?language .
        ?language rdf:type cidoc-crm:E56_Language .
        ?language rdfs:label ?Language}
  ?manifestation lrmoo:R4_embodies ?expression .
  ?item cidoc-crm:P137_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image .
   
      }

"""

results = graph.query(sparql_query)

for result in results:
    Identifier = result[0]
    Title = result[1]
    Description = result[2]
    Language = result[3]
    IIIF_Image = result[4]

    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Description:", Description)
    print("Language:", Language)
    print("IIIF Image:", IIIF_Image)


Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Description: Legenda
Language: None
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1489/full/full/0/iiifimg1492
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Description: Met lijsten van openbare gebouwen en gasthuizen en van straten en poorten
Language: None
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1223/full/full/0/default.jpg#iiifimg1224
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Description: Title in map
Language: Italian
IIIF Image: https://dbc.rug.nl/digital/iiif/Kaarten/1489/full/full/0/iiifimg1492


In [200]:
sparql_query = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX lrmoo: <http://iflastandards.info/ns/lrm/lrmoo/>
PREFIX cidoc-crm: <http://www.cidoc-crm.org/cidoc-crm/>
PREFIX crmdig: <http://www.cidoc-crm.org/extensions/crmdig/>

SELECT ?Creation_Date ?Cartographer ?Publication_Date ?Publisher ?Publication_Place ?Identifier ?Title ?IIIF_Image 
WHERE {
  ?map_creation rdf:type lrmoo:F28_Expression_Creation .
  ?map_publication rdf:type lrmoo:F30_Manifestation_Creation .
  ?map_creation lrmoo:R17_created ?expression .
  ?expression rdf:type lrmoo:F2_Expression .
  ?map_publication lrmoo:R24_created ?edition .
  ?edition rdf:type lrmoo:F3_Manifestation .
  ?edition lrmoo:R4_embodies ?expression .
  ?item cidoc-crm:P137_exemplifies ?manifestation .
  ?digitization crmdig:L1_digitized ?item .
  ?digitization crmdig:L20_has_created ?Digital_Image .
  ?Digital_Image cidoc-crm:P102_has_title ?title .
  ?title rdf:type cidoc-crm:E35_Title .
  ?title rdfs:label ?Title .
  ?Digital_Image cidoc-crm:P48_has_preferred_identifier ?Identifier .
  ?IIIF_Image cidoc-crm:P138_represents ?Digital_Image .
  OPTIONAL{?map_creation cidoc-crm:P4_has_time-span ?timespan1 . 
  ?timespan1 cidoc-crm:P82_at_some_time_within ?Creation_Date }
  OPTIONAL{?map_creation cidoc-crm:P14_carried_out_by ?mapmaker .
  ?mapmaker rdf:type cidoc-crm:E39_Actor .
  ?mapmaker rdfs:label ?Cartographer}
  OPTIONAL{?map_publication cidoc-crm:P4_has_time-span ?timespan2 . 
  ?timespan2 cidoc-crm:P82_at_some_time_within ?Publication_Date}
  OPTIONAL{?map_publication cidoc-crm:P14_carried_out_by ?publisher .
  ?publisher rdf:type cidoc-crm:E39_Actor .
  ?publisher rdfs:label ?Publisher}
  OPTIONAL{?map_publication cidoc-crm:P7_took_place_at ?publication_place .
  ?publication_place rdf:type cidoc-crm:E53_Place .
  ?publication_place rdfs:label ?Publication_Place}
      }

"""

results = graph.query(sparql_query)

unique_results = {}

for result in results:
    # Extracting values for each variable
    Identifier = result.get("Identifier")
    Title = result.get("Title")
    creation_date = result.get("Creation_Date")
    cartographer = result.get("Cartographer")
    publication_date = result.get("Publication_Date")
    publisher = result.get("Publisher")
    publication_place = result.get("Publication_Place")
    
    # Printing or processing the extracted values
    print("Identifier:", Identifier)
    print("Title:", Title)
    print("Creation Date:", creation_date)
    print("Cartographer:", cartographer)
    print("Publication Date:", publication_date)
    print("Publisher:", publisher)
    print("Publication Place:", publication_place)

Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1224
Title: Afbeeldinge der stadt Groningen met de omliggende fortressen : 1652
Creation Date: 1652
Cartographer: Egbert Haubois
Publication Date: 1694
Publisher: None
Publication Place: None
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1492
Title: Tweede stuk der beleegering van Bergen op Zoom : met de afbeeldinge der Franse attaque op de Linie en Roversfort
Creation Date: 1652
Cartographer: Egbert Haubois
Publication Date: 1694
Publisher: None
Publication Place: None
Identifier: http://cdm21053.contentdm.oclc.org/cdm/ref/collection/Kaarten/id/1498
Title: Grand theatre de la guerre sur les frontieres de France ÿ& d'Allemagne ou carte nouvelle du cours du Rhin et des païs circonvoisins contenant la Haute ÿ Basse Alsace, ...
Creation Date: 1652
Cartographer: Egbert Haubois
Publication Date: 1694
Publisher: None
Publication Place: None
Identifier: http://cdm21053.contentdm.oclc.or